In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class MySpider(scrapy.Spider):

    name = 'my_first_spider'

    def start_requests(self):
        urls = ['https://www.datacamp.com/courses-all']
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_front, headers={'User-Agent': 'XYZ/3.0'})

    def parse_front(self, response):
        # Narrow in on the course blocks
        course_blocks = response.css('div.css-3undxp-HitCard')
        # print(course_blocks)
        # Direct to the course links
        course_links = course_blocks.xpath('./a/@href')
        # Extract the links (as a list of strings)
        links_to_follow = course_links.extract()
        for link in links_to_follow:
            yield response.follow(url=link, callback=self.parse_pages, headers={'User-Agent': 'XYZ/3.0'})
    
    def parse_pages(self, response):
        # Direct to the course title text
        crs_title = response.css('h1.css-7ymucl-CourseShowPage::text')
        # crs_title = response.xpath('//h1[@class="css-7ymucl-CourseShowPage"]/text()')
        # Extract and clean the course title text
        crs_title_ext = crs_title.extract_first().strip()
        # Direct to the chapter description
        ch_desc = response.css('p.css-hnday4-CourseShowPage::text')
        # Extract and clean the chapter description
        ch_desc_ext = ch_desc.extract_first().strip()
        # Direct to the chapter titles text
        ch_titles = response.css('h3.css-172ju3k-Box::text')
        # ch_titles = resp.xpath('//h3[@class="css-172ju3k-Box"]/text()')
        # Extract and clean the chapter titles
        ch_titles_ext = [t.strip() for t in ch_titles.extract()]
        # Store this in our dictionary
        dc_dict[crs_title_ext] = {'Description': ch_desc_ext, 'Chapters': ch_titles_ext}


In [3]:
dc_dict = {}

process = CrawlerProcess()
process.crawl(MySpider)
process.start()

2023-07-23 19:22:10 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-07-23 19:22:10 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.9.13, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.6 (default, May  7 2023, 23:32:44) - [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform macOS-13.4.1-arm64-arm-64bit
2023-07-23 19:22:10 [scrapy.crawler] INFO: Overridden settings:
{}
2023-07-23 19:22:10 [py.warnings] WARNING: /Users/USUARIO/.virtualenvs/web_scrapping/lib/python3.9/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scr

In [4]:
dc_dict

{'Introduction to DAX in Power BI': {'Description': 'Enhance your Power BI knowledge, by learning the fundamentals of Data Analysis Expressions (DAX) such as calculated columns, tables, and measures.',
  'Chapters': ['Getting Started with DAX',
   'Context in DAX Formulas',
   'Working with Dates']},
 'Introduction to Importing Data in Python': {'Description': 'Learn to import data into Python from various sources, such as Excel, SQL, SAS and right from the web.',
  'Chapters': ['Introduction and flat files',
   'Importing data from other file types',
   'Working with relational databases in Python']},
 'Understanding Machine Learning': {'Description': 'An introduction to machine learning with no coding involved.',
  'Chapters': ['What is Machine Learning?',
   'Machine Learning Models',
   'Deep Learning']},
 'Introduction to Data Visualization with Seaborn': {'Description': 'Learn how to create informative and attractive visualizations in Python using the Seaborn library.',
  'Chapte

In [5]:
for course, course_items in dc_dict.items():
        print(f'Course: {course}')
        print(f'Description: {course_items["Description"]}')
        for i, chapter in enumerate(course_items['Chapters'], start=1):
            print(f'    Chapter {i}: {chapter}')
        print('')

Course: Introduction to DAX in Power BI
Description: Enhance your Power BI knowledge, by learning the fundamentals of Data Analysis Expressions (DAX) such as calculated columns, tables, and measures.
    Chapter 1: Getting Started with DAX
    Chapter 2: Context in DAX Formulas
    Chapter 3: Working with Dates

Course: Introduction to Importing Data in Python
Description: Learn to import data into Python from various sources, such as Excel, SQL, SAS and right from the web.
    Chapter 1: Introduction and flat files
    Chapter 2: Importing data from other file types
    Chapter 3: Working with relational databases in Python

Course: Understanding Machine Learning
Description: An introduction to machine learning with no coding involved.
    Chapter 1: What is Machine Learning?
    Chapter 2: Machine Learning Models
    Chapter 3: Deep Learning

Course: Introduction to Data Visualization with Seaborn
Description: Learn how to create informative and attractive visualizations in Python us